In [ ]:
import random
import textwrap
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# ------------------------------------------------------
# 0. Tiny built-in dataset: (label, text)
#    "spam" or "ham" (not spam)
# ------------------------------------------------------
data = [
    ("spam", "Congratulations! You have won a $1000 gift card. Click here to claim now!"),
    ("spam", "WIN a brand new iPhone. Limited time offer, act now!"),
    ("spam", "You have been selected for a lottery prize, reply with your bank details."),
    ("spam", "Cheap meds online!!! Buy now with 90% discount."),
    ("spam", "This is not a scam. Send your password to verify your account."),
    ("ham", "Hi John, can we move our meeting to tomorrow morning?"),
    ("ham", "Reminder: your dentist appointment is on Friday at 10am."),
    ("ham", "Thanks for sending over the report, I'll review it tonight."),
    ("ham", "Mum’s birthday is next week – shall we book a restaurant?"),
    ("ham", "Here are the notes from today’s lecture, let me know if you have questions."),
]

df = pd.DataFrame(data, columns=["label", "text"])
df["label_num"] = df["label"].map({"ham": 0, "spam": 1})

X = df["text"]
y = df["label_num"]

# ------------------------------------------------------
# 1. Train the AI spam detector
# ------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

model = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english", max_df=0.9)),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)

print("AI performance on test set:")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["ham", "spam"]))

# Put test data into a convenient structure for the game
test_emails = list(zip(X_test.tolist(), y_test.tolist()))

# ------------------------------------------------------
# 2. Game loop: Player vs AI
# ------------------------------------------------------
player_score = 0
ai_score = 0

WRAP_WIDTH = 70

def show_email(email_text):
    print("\n" + "-" * WRAP_WIDTH)
    print("New email received:")
    print("-" * WRAP_WIDTH)
    for line in textwrap.wrap(email_text, width=WRAP_WIDTH):
        print(line)
    print("-" * WRAP_WIDTH)

def ask_player():
    while True:
        ans = input("Is this spam or not? (s = spam, h = ham, q = quit): ").strip().lower()
        if ans in ["s", "h", "q"]:
            return ans
        print("Please type 's', 'h', or 'q'.")

print("\nWelcome to Inbox Duel: You vs AI spam detector!")
print("Rules:")
print("  +1 point if you are correct.")
print("  +1 point if the AI is correct.")
print("Try to beat the AI on these emails.\n")

random.shuffle(test_emails)

for email_text, true_label in test_emails:
    show_email(email_text)

    player_choice = ask_player()
    if player_choice == "q":
        break

    player_label = 1 if player_choice == "s" else 0

    ai_label = int(model.predict([email_text])[0])
    ai_prob = model.predict_proba([email_text])[0][1]

    true_str = "spam" if true_label == 1 else "ham"
    player_str = "spam" if player_label == 1 else "ham"
    ai_str = "spam" if ai_label == 1 else "ham"

    print(f"\nGround truth: {true_str.upper()}")
    print(f"Your answer:  {player_str.upper()}")
    print(f"AI answer:    {ai_str.upper()} (spam prob = {ai_prob:.2f})")

    if player_label == true_label:
        player_score += 1
        print("You: +1 point")
    if ai_label == true_label:
        ai_score += 1
        print("AI:  +1 point")

    print(f"Scores -> You: {player_score} | AI: {ai_score}\n")

print("\nGame over!")
print(f"Final scores -> You: {player_score} | AI: {ai_score}")
if player_score > ai_score:
    print("You beat the AI!")
elif player_score < ai_score:
    print("The AI wins this time.")
else:
    print("It's a draw.")



AI performance on test set:
              precision    recall  f1-score   support

         ham       0.00      0.00      0.00         2
        spam       0.33      1.00      0.50         1

    accuracy                           0.33         3
   macro avg       0.17      0.50      0.25         3
weighted avg       0.11      0.33      0.17         3


Welcome to Inbox Duel: You vs AI spam detector!
Rules:
  +1 point if you are correct.
  +1 point if the AI is correct.
Try to beat the AI on these emails.


----------------------------------------------------------------------
New email received:
----------------------------------------------------------------------
Hi John, can we move our meeting to tomorrow morning?
----------------------------------------------------------------------


C:\Users\Y.Baradaranshokouhi\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Y.Baradaranshokouhi\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\Y.Baradaranshokouhi\AppData\Roaming\Python\Python314\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war


Ground truth: HAM
Your answer:  SPAM
AI answer:    SPAM (spam prob = 0.57)
Scores -> You: 0 | AI: 0


----------------------------------------------------------------------
New email received:
----------------------------------------------------------------------
This is not a scam. Send your password to verify your account.
----------------------------------------------------------------------

Ground truth: SPAM
Your answer:  HAM
AI answer:    SPAM (spam prob = 0.57)
AI:  +1 point
Scores -> You: 0 | AI: 1


----------------------------------------------------------------------
New email received:
----------------------------------------------------------------------
Mum’s birthday is next week – shall we book a restaurant?
----------------------------------------------------------------------

Ground truth: HAM
Your answer:  SPAM
AI answer:    SPAM (spam prob = 0.57)
Scores -> You: 0 | AI: 1


Game over!
Final scores -> You: 0 | AI: 1
The AI wins this time.
